# Spam Message Classification using RNNs (LSTM, Bi-LSTM, GRU)

Personal project on natural language processing (NLP) using recurrent neural networks (RNNs). In this notebook, I build and compare deep learning models to classify SMS messages as either *spam* or *ham* (not spam).

The main goal here is to practice building models with LSTM, Bi-LSTM, and GRU layers using TensorFlow/Keras, as well as to explore basic text preprocessing and evaluation techniques.


## Problem Overview

This project focuses on building RNN-based models for text classification. The goal is to develop models that can detect whether a given SMS message is *spam* or *ham* (not spam), which is a typical binary classification task in NLP.

I'll go through the following steps:
- Data preprocessing
- Tokenization and padding
- Building LSTM, Bi-LSTM, and GRU models
- Training and evaluation

### References
- Inspired by: [Text Classification using LSTM, Bi-LSTM, and GRU](https://nzlul.medium.com/the-classification-of-text-messages-using-lstm-bi-lstm-and-gru-f79b207f90ad)
- Keras documentation on RNN layers: https://keras.io/api/layers/recurrent_layers/
- TensorFlow RNN guide: https://www.tensorflow.org/guide/keras/working_with_rnns


## Pacotes

In [ ]:
#  Load, explore and plot data
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
%matplotlib inline
#  Train test split
from sklearn.model_selection import train_test_split
#  Text pre-processing
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
#  Modeling
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional

## Pré-Processamento dos Dados

### Dataset

Base de dados de mensagens SMS de celulares, publicamente disponível na UCL datasets (https://archive.ics.uci.edu/dataset/228/sms+spam+collection). Pode também ser baixada de https://raw.githubusercontent.com/kenneth-lee-ch/SMS-Spam-Classification/master/spam.csv<br>
O dataset contém 5.574 mensagens rotuladas como *spam* ou não *spam* (*ham*).

A biblioteca Pandas foi usada para ler e manipular o dataset.

In [ ]:
#  read the dataset
df = pd.read_csv('./datasets/spam.csv', encoding='ISO-8859-1')
#  rename the columns
df = df[['v1','v2']]
df.rename(columns={'v1':'label', 'v2':'message'}, inplace=True)
#  show the first instances
df.head()

### Estatísticas sobre os Dados

Exibe um sumário das estatísticas para melhor entender os dados

In [ ]:
df.describe()

In [ ]:
df.groupby('label').describe().T

### Nuvem de Palavras

Visualiza as palavras mais frequentes em cada classe usando uma nuvem de palavras

In [ ]:
ham_msg  = df.loc[df['label'] == 'ham']
spam_msg = df.loc[df['label'] == 'spam']

In [ ]:
#  Nuvem de palavras da classe 'ham'
ham_msg_text = ' '.join(ham_msg['message'])
ham_msg_cloud = WordCloud(width =520, height =260, stopwords = STOPWORDS, max_font_size = 50, background_color = "black", colormap = 'Pastel1').generate(ham_msg_text)
plt.figure(figsize=(16,10))
plt.imshow(ham_msg_cloud, interpolation = 'bilinear')
plt.axis('off') #  turn off axis
plt.show()

Palavras mais frequentes da classe 'ham', de acordo com a nuvem de palavras: now, will, ok, today, Sorry etc.

Adicione abaixo um trecho de código para gerar a nuvem de palavras para os textos da classe 'spam'.<br>
Depois, verifique as palavras mais frequentes da classe.

In [ ]:
#  Nuvem de palavras da classe 'spam'
# # #  INICIE O CÓDIGO AQUI # # #  (6 ou mais linhas de código)
#  Nuvem de palavras da classe 'ham'
spam_msg_text = ' '.join(spam_msg['message'])
spam_msg_cloud = WordCloud(width =520, height =260, stopwords = STOPWORDS, max_font_size = 50, background_color = "black", colormap = 'Pastel1').generate(spam_msg_text)
plt.figure(figsize=(16,10))
plt.imshow(spam_msg_cloud, interpolation = 'bilinear')
plt.axis('off') #  turn off axis
plt.show()
# # #  TERMINE O CÓDIGO AQUI # # # 

Palavras mais frequentes da classe 'spam', de acordo com a nuvem de palavras: FREE, call, URGENT, mobile, etc

### Balanceamento dos Dados

Como veremos abaixo, o dataset está muito desbalanceado. Existem muito mais mensagens na classe 'ham' do que na classe 'spam'.<br>
O treinamento de modelos de aprendizagem de máquina a partir de datasets muito desbalanceados pode gerar um viés, levando o modelo a predizer a classe mais frequente.<br>

Existem algumas abordagens para tratar o problema de dados desbalanceados, dentre elas: escolher métricas de avaliação mais apropriadas, resampling (oversampling and undersampling), Synthetic Minority Oversampling Technique (SMOTE), BalancedBaggingClassifier, Threshold moving.<br>

Neste notebook, nós usaremos o método undersampling (subamostragem) para manusear os dados desbalanceados. A técnica consiste em subamostrar a classe majoritária de forma aleatória e uniforme, escolhendo, aproximadamente, o mesmo número de instâncias da classe minoritária. Isso pode potenciamente conduzir a perda de informação, mas se os exemplos da classe majoritária estiverem próximos uns aos outros, esse método pode levar a bons resultados.

In [ ]:
#  Distribuição das mensagens em 'ham' e 'spam'
plt.figure(figsize=(8,6))
sns.countplot(df.label)
plt.title('The distribution of ham and spam messages')

In [ ]:
#  downsample the ham msg
ham_msg_df = ham_msg.sample(n = len(spam_msg), random_state = 44)

In [ ]:
msg_df = pd.concat([ham_msg_df, spam_msg])
msg_df.head()

In [ ]:
msg_df.tail()

In [ ]:
#  Nova distribuição
plt.figure(figsize=(8,6))
sns.countplot(msg_df.label)
plt.title('The distribution of ham and spam messages')

### Pré-processamento do Texto

Cria duas colunas no *dataframe*: uma para armazenar o comprimento de cada mensagem de texto e outra para armazenar o rótulo da classe convertido para um valor numérico (0: ham, 1: spam).

In [ ]:
#  Get length column for each text
msg_df['text_length'] = msg_df['message'].apply(len)
#  Get the converted numeric label of the data
msg_df['msg_type'] = msg_df['label'].map({'ham':0, 'spam':1})
msg_df.head()

In [ ]:
msg_df.tail()

### Divisão dos Dados em Treino e Teste

Os dados são divididos, aletatoriamente, em 80% para treino e 20% para teste.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(msg_df['message'], msg_df['msg_type'], test_size=0.2, random_state=434)

In [ ]:
X_train.tail()

In [ ]:
y_train.tail()

### Tokenização

Os textos das mensagens precisam ser convertidos para uma representação numérica, para que o modelo possa entendê-los.

A API Tokenizer do TensorFlow divide as sentenças em palavras e as codifica em números inteiros.

O Tokenizer executará os seguintes passos de pré-processamento: tokeniza a nível de palavras, remove os termos de pontuação, converte todas as palavras para minúsculas, converte todas as palavras para números inteiros. Os seguintes parâmetros foram definidos:
- num_words: número de palavras únicas (vocabulário)
- oov_token: token usado para substituir palavras que não estiverem no vocabulário

In [ ]:
#  Defining pre-processing parameters
max_len = 50 
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' #  out of vocabulary token
vocab_size = 500

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, 
                      char_level = False,
                      oov_token = oov_tok)
tokenizer.fit_on_texts(X_train)

In [ ]:
#  Get the word_index
word_index = tokenizer.word_index
total_words = len(word_index)
total_words

Em seguida, cada sentença é representada por uma sequência de números usando o método texts_to_sequences do objeto Tokenizer.

Depois, cada setença é completada com o token de 'pad' ou truncada para que todas tenham o mesmo comprimento.

Os parâmetros são:
- maxlen: tamanho máximo de todas as sequências. O valor *default* é o comprimento da sentença mais longa.
- padding: 'pre' ou 'post' (*default*). Completa com tokens 'pad' antes ('pre') ou depois ('post') de cada sequencia.
- truncating: 'pre' ou 'post' (*default*). Se o tamanho de uma sentença for maior do que o valor de 'maxlen', ela será truncada para 'maxlen'. A opção 'pre' trunca no início e 'post' trunca no final da sequência.

In [ ]:
#  Dados de treino
training_sequences = tokenizer.texts_to_sequences(X_train)
training_padded = pad_sequences(training_sequences,
                                maxlen = max_len,
                                padding = padding_type,
                                truncating = trunc_type)

In [ ]:
#  Dados de teste
testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences,
                               maxlen = max_len,
                               padding = padding_type,
                               truncating = trunc_type)

In [ ]:
#  Formato dos tensores de treino e teste
print('Shape of training tensor: ', training_padded.shape)
print('Shape of testing tensor: ', testing_padded.shape)

## Configuração do Modelo de Classificação

Define a arquitetura do modelo de classificação. O modelo usa uma Rede Neural Recorrente do tipo LSTM (Long Short Term Memory) Bidirecional.

O modelo sequencial Keras permite a adição de camadas em uma sequência. Você deve adicionar as seguintes camadas em sequência:
- Camada de Embedding, que mapeia cada palavra para um vetor N-dimensional de número reais. O 'embedding_dim' é o tamanho do vetor, nesse caso, 16. Como a camada de embedding é a primeira camada oculta do modelo, a camada de entrada deve ser definida por input_length = max_len.
- Camada LSTM bidirecional, com 128 unidades.
- Camada de Dropout com uma fração de drop = 0.2
- Camada Densa (camada de classificação binária) com uma unidade e função de ativação sigmoid.

In [ ]:
embedding_dim = 16  #  tamanho do embedding (vetor) de palavras
n_lstm = 128        #  número de unidades (dimensionalidade da saída)
drop_lstm = 0.2     #  fração das unidades para drop

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size,
                    embedding_dim,
                    input_length = max_len))
# # #  INICIE O CÓDIGO AQUI # # #  (3 linhas de código)
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
# # #  TERMINE O CÓDIGO AQUI # # # 

Exibe um sumário do modelo

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics=['accuracy'])

## Treinamento do Modelo

Treina o modelo de classificação usando o método 'fit'.

EarlyStopping (monitor='val_loss', patience=2) define que o método vai monitorar a perda nos dados de validação, e se a perda não for melhorada após 2 épocas, então o modelo de treinamento é finalizado. Essa técnica ajuda a evitar problemas de *overfitting*.

In [ ]:
num_epochs = 30
early_stop = EarlyStopping(monitor = 'val_loss',
                           patience = 2)
history = model.fit(training_padded,
                    y_train,
                    epochs = num_epochs,
                    validation_data = (testing_padded, y_test),
                    callbacks = [early_stop],
                    verbose = 2)

Plota gráficos de acurácia e perda

In [ ]:
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

## Avaliação do Modelo

Avalia o desempenho do modelo no conjunto de teste.

In [ ]:
test_loss, test_acc = model.evaluate(testing_padded, y_test)
print(f"LSTM model loss: {test_loss} " )
print(f"LSTM model accuracy: {test_acc*100:0.2f}%" )

## Predição

Prediz a saída de novas mensagens

In [ ]:
predict_msg = ["Have friends and colleagues who could benefit from these weekly updates? Send them to this link to subscribe",
               "Call me","Get this subscription for free!"]

In [ ]:
def predict_spam(predict_msg):
  new_seq = tokenizer.texts_to_sequences(predict_msg)
  padded = pad_sequences(new_seq,
                         maxlen = max_len,
                         padding = padding_type,
                         truncating = trunc_type)
  return(model.predict(padded))

In [ ]:
#  'ham' se predict < 0.5 senão 'spam'
predict_spam(predict_msg)

## Desafio 1

Modifique a configuração do modelo de classificação de forma a usar três camadas de LSTM Bidirecional, em vez de apenas uma, como foi feito do código acima.

Você vai precisar replicar todas as células do notebook, desde a Configuração do Modelo de Classificação, fazendo as devidas adaptações para a nova configuração.

A saída esperada para o model.sumary() da nova arquitetura é:

Adicione abaixo a nova sequência de código.

In [ ]:
# # #  INICIE O CÓDIGO AQUI # # #  (várias linhas de código / várias células)
model = Sequential([Embedding(vocab_size, embedding_dim,input_length = max_len),
                    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
                    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
                    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
                    tf.keras.layers.Dropout(0.2),
                    tf.keras.layers.Dense(1, activation='sigmoid')
                    ])


In [ ]:
model.summary()

#### Configuração do Modelo de Classificação

In [ ]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics=['accuracy'])

#### Treinamento do Modelo

In [ ]:
num_epochs = 30
early_stop = EarlyStopping(monitor = 'val_loss',
                           patience = 2)
history = model.fit(training_padded,
                    y_train,
                    epochs = num_epochs,
                    validation_data = (testing_padded, y_test),
                    callbacks = [early_stop],
                    verbose = 2)

#### Avaliação do Modelo

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
triple_lstm_test_loss, triple_lstm_test_acc = model.evaluate(testing_padded, y_test)
print(f"LSTM model loss: {triple_lstm_test_loss} " )
print(f"LSTM model accuracy: {triple_lstm_test_acc*100:0.2f}%" )

#### Predição

In [ ]:
predict_msg = ["Have friends and colleagues who could benefit from these weekly updates? Send them to this link to subscribe",
               "Call me","Get this subscription for free!"]

#  'ham' se predict < 0.5 senão 'spam'
predict_spam(predict_msg)

In [ ]:
# # #  TERMINE O CÓDIGO AQUI # # # 

## Desafio 2

Modifique a configuração do modelo de classificação de forma a usar uma Gated Recurrent Unit (GRU) Bidirecional, em vez de uma LSTM.

Você vai precisar replicar todas as células do notebook, desde a Configuração do Modelo de Classificação, fazendo as devidas adaptações para a nova configuração.

A saída esperada para o model.sumary() da nova arquitetura é:

Adicione abaixo a nova sequência de código.

In [ ]:
# # #  INICIE O CÓDIGO AQUI # # #  (várias linhas de código / várias células)

#### Configuração do Modelo de Classificação

In [ ]:
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    Bidirectional(GRU(128)),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])


In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics=['accuracy'])

#### Treinamento do Modelo

In [ ]:
num_epochs = 30
early_stop = EarlyStopping(monitor = 'val_loss',
                           patience = 2)
history = model.fit(training_padded,
                    y_train,
                    epochs = num_epochs,
                    validation_data = (testing_padded, y_test),
                    callbacks = [early_stop],
                    verbose = 2)

#### Avaliação do Modelo

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
gru_test_loss, gru_test_acc = model.evaluate(testing_padded, y_test)
print(f"LSTM model loss: {gru_test_loss} " )
print(f"LSTM model accuracy: {gru_test_acc*100:0.2f}%" )

#### Predição

In [ ]:
predict_msg = ["Have friends and colleagues who could benefit from these weekly updates? Send them to this link to subscribe",
               "Call me","Get this subscription for free!"]

#  'ham' se predict < 0.5 senão 'spam'
predict_spam(predict_msg)

In [ ]:
# # #  TERMINE O CÓDIGO AQUI # # # 

## Comparação dos Resultados

Adicione abaixo uma tabela com a comparação dos resultados obtidos pelas três configurações efetuadas neste notebook.<br>
Qual configuração obteve o melhor resultados?

In [ ]:
# # #  ADICIONE A TABELA DE RESULTADOS AQUI # # # 

In [ ]:
results = {
    'Modelo': ['LSTM', 'Stacked LSTM (3 layers)', 'GRU'],
    'Loss': [test_loss, triple_lstm_test_loss, gru_test_loss],
    'Acurácia': [test_acc, triple_lstm_test_acc, gru_test_acc]
}

df_resultados = pd.DataFrame(results)

df_resultados = df_resultados.sort_values(by='Acurácia', ascending=False)

df_resultados

O modelo LSTM de 3 camadas obteve a melhor acurácia, sendo 1,1% melhor que o modelo GRU. Entretanto, o modelo GRU apresenta menor custo computacional e o menor loss, o que pode ser positivo dependendo de sua aplicação.

# Fim

Parabéns! Você efetuou todos os passos para criar modelos baseados em Redes Neurais Recorrentes para a tarefa de classificação de textos.

-------------------------------------------